In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
import scipy.stats as st
from datetime import date
from functools import reduce
import json
import ctypes
import seaborn as sns
import datetime

In [2]:
df = pd.read_csv('../Data/SUB-IP-EST2019-ANNRNK-1.csv')
# df['City Name'] = df['Geographic Area'].str.split(', ').apply(lambda x: x[0])
df.dropna()
df['2018'] = df['2018'].astype('int64')
# df['Geographic Area'] = df['Geographic Area'].astype('str')
df.dtypes

Geographic Area    object
2018                int64
dtype: object

In [3]:
df['City Name'] = df['Geographic Area'].str.split(', ').apply(lambda x: x[0])
df['State Name'] = df['Geographic Area'].str.split(', ').apply(lambda x: x[-1])
df['City Name'] = df['City Name'].str.rstrip('city')
df.rename(columns = {"2018":"Population"}, inplace = True)
df.drop(columns = ["Geographic Area"], inplace = True)
df['City Name'] = df['City Name'].astype(str)
df.reset_index()

,index,Population,City Name,State Name
0,0,8390081,New York,New York
1,1,3977596,Los Angeles,California
2,2,2701423,Chicago,Illinois
3,3,2318573,Houston,Texas
4,4,1654675,Phoenix,Arizona
...,...,...,...,...
783,783,50012,Lakewood,Ohio
784,784,49354,Troy,New York
785,785,48310,Saginaw,Michigan
786,786,48012,Niagara Falls,New York


In [4]:
df

,Population,City Name,State Name
0,8390081,New York,New York
1,3977596,Los Angeles,California
2,2701423,Chicago,Illinois
3,2318573,Houston,Texas
4,1654675,Phoenix,Arizona
...,...,...,...
783,50012,Lakewood,Ohio
784,49354,Troy,New York
785,48310,Saginaw,Michigan
786,48012,Niagara Falls,New York


In [5]:
# code coppied from https://gist.github.com/rogerallen/1583593
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}

In [6]:
df['State Name'] = df['State Name'].map(us_state_abbrev).fillna(df['State Name'])
df

,Population,City Name,State Name
0,8390081,New York,NY
1,3977596,Los Angeles,CA
2,2701423,Chicago,IL
3,2318573,Houston,TX
4,1654675,Phoenix,AZ
...,...,...,...
783,50012,Lakewood,OH
784,49354,Troy,NY
785,48310,Saginaw,MI
786,48012,Niagara Falls,NY


In [7]:
us_state_abbrev = {state: abbrev for abbrev, state in us_state_abbrev.items()}
df['State'] = df['State Name'].map(us_state_abbrev)
df

,Population,City Name,State Name,State
0,8390081,New York,NY,New York
1,3977596,Los Angeles,CA,California
2,2701423,Chicago,IL,Illinois
3,2318573,Houston,TX,Texas
4,1654675,Phoenix,AZ,Arizona
...,...,...,...,...
783,50012,Lakewood,OH,Ohio
784,49354,Troy,NY,New York
785,48310,Saginaw,MI,Michigan
786,48012,Niagara Falls,NY,New York


In [8]:
df.drop(columns = ['State'], inplace = True)
df.rename(columns = {"City Name":"City"}, inplace = True)
pop_df = df

In [9]:
df.loc[df['City'] == 'Indianapolis']

,Population,City,State Name


In [10]:
nba_df = pd.read_csv('../Data/MLB-revenue-2018.csv')
nba_df

,Team,City,State/Province,Revenue
0,Los Angeles Dodgers,Los Angeles,CA,3000000000
1,Chicago Cubs,Chicago,IL,2900000000
2,San Francisco Giants,San Francisco,CA,2850000000
3,Boston Red Sox,Boston,MA,2800000000
4,New York Mets,New York,NY,2100000000
5,St. Louis Cardinals,Saint Louis,MO,1900000000
6,Los Angeles Angels,Los Angeles,CA,1800000000
7,Philadelphia Phillies,Philadelphia,PA,1700000000
8,Washington Nationals,Washington,DC,1675000000
9,Houston Astros,Houston,TX,1650000000


In [11]:
pop_df['City'] = pop_df['City'].str.strip()
nba_df['City'] = nba_df['City'].str.strip()
nba_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29 entries, 0 to 28
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Team            29 non-null     object
 1   City            29 non-null     object
 2   State/Province  29 non-null     object
 3   Revenue         29 non-null     int64 
dtypes: int64(1), object(3)
memory usage: 1.0+ KB


In [12]:
merged_nba_df = pd.merge(nba_df, pop_df, how='inner', on = ['City'])

merged_nba_df

In [13]:
merged_nba_df

,Team,City,State/Province,Revenue,Population,State Name
0,Los Angeles Dodgers,Los Angeles,CA,3000000000,3977596,CA
1,Los Angeles Angels,Los Angeles,CA,1800000000,3977596,CA
2,Chicago Cubs,Chicago,IL,2900000000,2701423,IL
3,Chicago White Sox,Chicago,IL,1500000000,2701423,IL
4,San Francisco Giants,San Francisco,CA,2850000000,880696,CA
5,Boston Red Sox,Boston,MA,2800000000,691147,MA
6,New York Mets,New York,NY,2100000000,8390081,NY
7,Philadelphia Phillies,Philadelphia,PA,1700000000,1583592,PA
8,Washington Nationals,Washington,DC,1675000000,701547,DC
9,Houston Astros,Houston,TX,1650000000,2318573,TX


In [14]:
# merged_nba_df = merged_nba_df.loc[merged_nba_df['State Name'] != 'ME']
# merged_nba_df.reset_index()
# merged_nba_df
# cleaned_merged_nba_df = merged_nba_df.drop(merged_nba_df.index[14])
cleaned_merged_nba_df.reset_index()
# merged_nba_df.drop(columns = ["index"], inplace = True)
cleaned_merged_nba_df

NameError: name 'cleaned_merged_nba_df' is not defined

In [15]:
# df.to_csv("pop_cities.csv", encoding="utf-8", index=False)

In [16]:
income_df = pd.read_csv('../Data/lapi1120msa-1.csv')
income_df.dropna()
income_df['Income'] = income_df['2018.00'].astype('int64')
income_df = income_df.dropna(how='any')
income_df.drop(columns = ['2018.00'], inplace = True)
income_df.reset_index()
income_df

,Metropolitan Statistical Areas,Income
0,"Abilene, TX",43585
1,"Akron, OH",49556
2,"Albany, GA",38034
3,"Albany-Lebanon, OR",43327
4,"Albany-Schenectady-Troy, NY",58443
...,...,...
379,"Yakima, WA",43905
380,"York-Hanover, PA",49888
381,"Youngstown-Warren-Boardman, OH-PA",42255
382,"Yuba City, CA",43662


In [17]:
income_df['City Name'] = income_df['Metropolitan Statistical Areas'].str.split(', ').apply(lambda x: x[0])
income_df['State Name'] = income_df['Metropolitan Statistical Areas'].str.split(', ').apply(lambda x: x[-1])
income_df.drop(columns = ['Metropolitan Statistical Areas'], inplace = True)
income_df.reset_index()


,index,Income,City Name,State Name
0,0,43585,Abilene,TX
1,1,49556,Akron,OH
2,2,38034,Albany,GA
3,3,43327,Albany-Lebanon,OR
4,4,58443,Albany-Schenectady-Troy,NY
...,...,...,...,...
379,379,43905,Yakima,WA
380,380,49888,York-Hanover,PA
381,381,42255,Youngstown-Warren-Boardman,OH-PA
382,382,43662,Yuba City,CA


In [18]:
# code found at https://riptutorial.com/pandas/example/25462/split--reshape--csv-strings-in-columns-into-multiple-rows--having-one-element-per-row

reshaped_city = \
(income_df.set_index(income_df.columns.drop('City Name',1).tolist())
   ['City Name'].str.split('-', expand=True)
   .stack()
   .reset_index()
   .rename(columns={0:'City Name'})
   .loc[:, income_df.columns]
)

reshaped_city

,Income,City Name,State Name
0,43585,Abilene,TX
1,49556,Akron,OH
2,38034,Albany,GA
3,43327,Albany,OR
4,43327,Lebanon,OR
...,...,...,...
579,42255,Youngstown,OH-PA
580,42255,Warren,OH-PA
581,42255,Boardman,OH-PA
582,43662,Yuba City,CA


In [19]:
reshaped_state = \
(reshaped_city.set_index(reshaped_city.columns.drop('State Name',1).tolist())
   ['State Name'].str.split('-', expand=True)
   .stack()
   .reset_index()
   .rename(columns={0:'State Name'})
   .loc[:, reshaped_city.columns]
)

reshaped_state

,Income,City Name,State Name
0,43585,Abilene,TX
1,49556,Akron,OH
2,38034,Albany,GA
3,43327,Albany,OR
4,43327,Lebanon,OR
...,...,...,...
683,42255,Warren,PA
684,42255,Boardman,OH
685,42255,Boardman,PA
686,43662,Yuba City,CA


In [20]:
city_income_df = reshaped_state

In [21]:
city_income_df.rename(columns = {"City Name":"City"}, inplace = True)
city_income_df

,Income,City,State Name
0,43585,Abilene,TX
1,49556,Akron,OH
2,38034,Albany,GA
3,43327,Albany,OR
4,43327,Lebanon,OR
...,...,...,...
683,42255,Warren,PA
684,42255,Boardman,OH
685,42255,Boardman,PA
686,43662,Yuba City,CA


In [22]:
merged_nba_df['City'] = merged_nba_df['City'].str.strip()
city_income_df['City'] = city_income_df['City'].str.strip()
pop_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 788 entries, 0 to 787
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Population  788 non-null    int64 
 1   City        788 non-null    object
 2   State Name  788 non-null    object
dtypes: int64(1), object(2)
memory usage: 18.6+ KB


In [23]:
merged_in_pop_nba_df = pd.merge(merged_nba_df, city_income_df, how='inner', on = ['City'])
merged_in_pop_nba_df

,Team,City,State/Province,Revenue,Population,State Name_x,Income,State Name_y
0,Los Angeles Dodgers,Los Angeles,CA,3000000000,3977596,CA,63886,CA
1,Los Angeles Angels,Los Angeles,CA,1800000000,3977596,CA,63886,CA
2,Chicago Cubs,Chicago,IL,2900000000,2701423,IL,61642,IL
3,Chicago Cubs,Chicago,IL,2900000000,2701423,IL,61642,IN
4,Chicago Cubs,Chicago,IL,2900000000,2701423,IL,61642,WI
5,Chicago White Sox,Chicago,IL,1500000000,2701423,IL,61642,IL
6,Chicago White Sox,Chicago,IL,1500000000,2701423,IL,61642,IN
7,Chicago White Sox,Chicago,IL,1500000000,2701423,IL,61642,WI
8,San Francisco Giants,San Francisco,CA,2850000000,880696,CA,100236,CA
9,Boston Red Sox,Boston,MA,2800000000,691147,MA,78923,MA


In [24]:
merged_in_pop_nba_df = merged_in_pop_nba_df.drop_duplicates(subset=['Team'])
merged_in_pop_nba_df.reset_index()


,index,Team,City,State/Province,Revenue,Population,State Name_x,Income,State Name_y
0,0,Los Angeles Dodgers,Los Angeles,CA,3000000000,3977596,CA,63886,CA
1,1,Los Angeles Angels,Los Angeles,CA,1800000000,3977596,CA,63886,CA
2,2,Chicago Cubs,Chicago,IL,2900000000,2701423,IL,61642,IL
3,5,Chicago White Sox,Chicago,IL,1500000000,2701423,IL,61642,IL
4,8,San Francisco Giants,San Francisco,CA,2850000000,880696,CA,100236,CA
5,9,Boston Red Sox,Boston,MA,2800000000,691147,MA,78923,MA
6,11,New York Mets,New York,NY,2100000000,8390081,NY,76565,NY
7,14,Philadelphia Phillies,Philadelphia,PA,1700000000,1583592,PA,64228,PA
8,18,Washington Nationals,Washington,DC,1675000000,701547,DC,72340,DC
9,22,Houston Astros,Houston,TX,1650000000,2318573,TX,57154,TX


In [25]:
merged_in_pop_nba_df.sort_values("Team")

,Team,City,State/Province,Revenue,Population,State Name_x,Income,State Name_y
33,Arizona Diamondbacks,Phoenix,AZ,1210000000,1654675,AZ,46539,AZ
23,Atlanta Braves,Atlanta,GA,1625000000,498183,GA,53188,GA
34,Baltimore Orioles,Baltimore,MD,1200000000,602443,MD,62013,MD
9,Boston Red Sox,Boston,MA,2800000000,691147,MA,78923,MA
2,Chicago Cubs,Chicago,IL,2900000000,2701423,IL,61642,IL
5,Chicago White Sox,Chicago,IL,1500000000,2701423,IL,61642,IL
46,Cincinnati Reds,Cincinnati,OH,1010000000,302277,OH,54055,OH
38,Cleveland Indians,Cleveland,OH,1045000000,383214,OH,38594,TN
37,Colorado Rockies,Denver,CO,1100000000,716265,CO,64690,CO
32,Detroit Tigers,Detroit,MI,1225000000,672977,MI,52572,MI


In [26]:
merged_in_pop_nba_df.rename(columns = {"State/Province":"State"}, inplace = True)
merged_in_pop_nba_df.drop(columns = ["State Name_x"], inplace = True)
merged_in_pop_nba_df.drop(columns = ["State Name_y"], inplace = True)
merged_in_pop_nba_df

C:\Users\camer\anaconda3\envs\PythonData\lib\site-packages\pandas\core\frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
C:\Users\camer\anaconda3\envs\PythonData\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Team,City,State,Revenue,Population,Income
0,Los Angeles Dodgers,Los Angeles,CA,3000000000,3977596,63886
1,Los Angeles Angels,Los Angeles,CA,1800000000,3977596,63886
2,Chicago Cubs,Chicago,IL,2900000000,2701423,61642
5,Chicago White Sox,Chicago,IL,1500000000,2701423,61642
8,San Francisco Giants,San Francisco,CA,2850000000,880696,100236
9,Boston Red Sox,Boston,MA,2800000000,691147,78923
11,New York Mets,New York,NY,2100000000,8390081,76565
14,Philadelphia Phillies,Philadelphia,PA,1700000000,1583592,64228
18,Washington Nationals,Washington,DC,1675000000,701547,72340
22,Houston Astros,Houston,TX,1650000000,2318573,57154


In [27]:
merged_in_pop_nba_df.to_csv("mlb_pop_income_city.csv", encoding="utf-8", index=False)

In [ ]:
pop_df['City Name'] = pop_df['City Name'].str.strip()
income_df['City Name'] = income_df['City Name'].str.strip()
pop_df.info()

In [ ]:
merged_df = pd.merge(pop_df, income_df, how='inner', on = ['City Name'])
merged_df.drop(columns = ['State Name_x'], inplace = True)

merged_df

In [ ]:
merged_df['State'] = merged_df['State Name_y']
merged_df['City'] = merged_df['City Name']
merged_df.drop(columns = ['State Name_y'], inplace = True)
merged_df = merged_df.dropna(how='any')
column_names = ["Population", "Income", "City", "State"]
merged_df = merged_df.reindex(columns=column_names)

income_df.reset_index()


In [ ]:
merged_df.tail(50)

In [ ]:
merged_df = pd.DataFrame(merged_df)
result_df = merged_df.drop_duplicates(subset=['City'])
result_df.reset_index()
result_df

In [ ]:
result_df.loc[merged_df['City Name'] == 'Cincinnati']

In [ ]:
result_df.to_csv("pop_income_city.csv", encoding="utf-8", index=False)

In [ ]:
# only 2 cities have the exact same income 
